In [1]:
import ast
import numpy as np
import pandas as pd

from opportunistic_planning import processing, prediction

In [11]:
df = pd.read_csv('all_task_environments_list_2022-03-28.csv', header=0)

In [13]:
def generate_distances_dict(data, use_string_for_seq=False, 
                            dimensions=[[1, 'x'], [1, 'y'], [1, 'z'], [2, 'xy'], [2, 'xz'], [2, 'yz'], [3, 'xyz']]):
    '''
    Calculate all object distances in all dimensions (e.g., xy, xyz) to reduce computational effort
    in main optimization function (calculate_prediction_error).
    
    Parameters
    ----------
    data : dataframe with object information
    dimensions : list of dimensions to be considered, optional
                The default is [[1, 'x'], [1, 'y'], [1, 'z'], [2, 'xy'], [2, 'xz'], [2, 'yz'], [3, 'xyz']].

    Returns
    -------
    distances_dict : dictionary of all object distances for all dimensions

    '''
    distances_dict = {}
    
    for dim in dimensions:
        dimension = dim[1]
        distances_dict[dimension] = {}
    
        for row in range(0,len(data)):
            #objects = list(data.at[row,'sequence'])
            
            if use_string_for_seq == True:
                objects = list(data.at[row, 'sequence'])
            else:
                objects = [elem for elem in data.at[row, 'sequence'].split(',')]
            
            ID = str(data.at[row,'ID'])
            #print(ID)
            start_coordinates = list(ast.literal_eval(data.at[row,'start_coordinates']))
            coordinates = {key: ast.literal_eval(value) for key, value in
                       (elem.split(': ') for elem in data.at[row,'coordinates'].split(';'))}
    
            distances_dict[dimension][ID] = {}
            
            new_coords, new_start_coords = filter_for_dimension(dim, coordinates, start_coordinates)
    
            for pos in new_start_coords:
                try:
                    position = tuple(pos)
                except TypeError:
                    position = str(pos)
                
                distances_dict[dimension][ID][position] = {}
                
                for obj in objects:
                    if obj not in distances_dict[dimension][ID][position]:
                        distances_dict[dimension][ID][position][obj] = np.linalg.norm(np.array(pos) -
                                                                     np.array(new_coords[obj]))
                
    return distances_dict

In [7]:
def filter_for_dimension(dimension, coordinates, start_coordinates):
    '''
    Filter coordinates and start coordinates for given dimension (e.g., xyz -> xy).

    Parameters
    ----------
    dimension : list of [int, str]
        Dimension for which to adapt coordinates (default before filtering: 3D).
    coordinates : dictionary
        Coordinates of objects in 3D.
    start_coordinates : list
        List of start coordinates where subject is standing before next picking_up action
        in 3D.

    Returns
    -------
    new_coords : dictionary
        Dictionary with filtered coordinates.
    new_start_coords : list
        List with filtered start coordinates.

    '''
    
    new_coords =  {}
    new_start_coords = []
    
    if dimension[0] == 3:  # no changes if 3D
        new_coords = coordinates
        new_start_coords = start_coordinates

    elif dimension[0] == 2:  # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coordinates.items()}
            new_start_coords = [x[:-1] for x in start_coordinates]

        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coordinates]

            for key, value in coordinates.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value

        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coordinates.items()}
            new_start_coords = [x[1:] for x in start_coordinates]

    elif dimension[0] == 1:  # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coordinates.items()}
            new_start_coords = [x[0] for x in start_coordinates]

        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coordinates.items()}
            new_start_coords = [x[1] for x in start_coordinates]

        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coordinates.items()}
            new_start_coords = [x[2] for x in start_coordinates]
            
    return new_coords, new_start_coords

In [3]:
df

,ID,sequence,coordinates,start_coordinates,strong_k,mid_k,food_k,containment,error
0,a1,"plate,plate_small,cup,glass,knife,spoon_small","plate: (0.008034,0.957082,0.6890539999999999);...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",tray,"plate,plate_small",0,0,0.723
1,a3,"cup,glass,wineglass,plate,cup,fork,knife,spoon","cup: (-0.525007,0.923283,0.438);glass: (-0.595...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",tray,"plate,plate_small",0,0,0.785
2,a5,"knife,fork,spoon,fork,knife,spoon,plate,winegl...","knife: (-0.07699199999999999,0.733425,0.531662...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",tray,"plate,plate_small",0,0,0.806
3,a11,"plate,fork,knife,spoon,wineglass,knife,fork,sp...","plate: (0.008034,0.957082,0.6890539999999999);...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",tray,"plate,plate_small",0,0,0.862
4,a13,"wineglass,plate,tray,glass,knife,fork,spoon,pl...","wineglass: (-0.196022,1.35572,0.466005);plate:...","[-0.451354,-0.413918,0.156247],[0.513,-0.531,0...",tray,"plate,plate_small",0,0,0.806
...,...,...,...,...,...,...,...,...,...
185,v8,"drink,drink,spoon,cereal,bowl,glass","drink: (0,4,1);spoon: (0,2,2);cereal: (0,1,3);...","[1,3,2],[0,4,1],[0,4,1],[0,2,2],[0,1,3],[2,3,2]",0,0,0,"drink,spoon,cereal,bowl,glass",0.723
186,v9,"drink,drink,spoon,cereal,glass,bowl","drink: (0,4,1);spoon: (0,2,2);cereal: (0,1,3);...","[1,3,2],[0,4,1],[0,4,1],[0,2,2],[0,1,3],[0,2,2]",0,0,0,"drink,spoon,cereal,bowl,glass",0.723
187,v10,"drink,drink,spoon,glass,cereal,bowl","drink: (0,4,1);spoon: (0,2,2);cereal: (0,1,3);...","[1,3,2],[0,4,1],[0,4,1],[0,2,2],[0,2,2],[0,1,3]",0,0,0,"drink,spoon,cereal,bowl,glass",0.723
188,v11,"drink,drink,spoon,glass,cereal,bowl","drink: (0,4,1);spoon: (0,2,2);cereal: (0,1,3);...","[1,3,2],[0,4,1],[0,4,1],[0,2,2],[0,2,2],[0,1,3]",0,0,0,"drink,spoon,cereal,bowl,glass",0.723


In [19]:
distances_dict = processing.generate_distances_dict(df)

In [15]:
results_sum = processing.calculate_prediction_error(df, distances_dict, 
                                                    error_function='prequential',
                                                   n=10, dimensions=[[2, 'xy']])

/home/zauri/files/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [16]:
results_sum

,"c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.1; k: 0.0,0.1,1.1; xy","c: 1.2; k: 0.0,0.1,1.1; xy","c: 1.3; k: 0.0,0.1,1.1; xy","c: 1.4; k: 0.0,0.1,1.1; xy","c: 1.5; k: 0.0,0.1,1.1; xy","c: 1.6; k: 0.0,0.1,1.1; xy","c: 1.7; k: 0.0,0.1,1.1; xy","c: 1.8; k: 0.0,0.1,1.1; xy","c: 1.9; k: 0.0,0.1,1.1; xy",...,"c: 1.2; k: 0.8,0.9,1.9; xy","c: 1.3; k: 0.8,0.9,1.9; xy","c: 1.4; k: 0.8,0.9,1.9; xy","c: 1.5; k: 0.8,0.9,1.9; xy","c: 1.6; k: 0.8,0.9,1.9; xy","c: 1.7; k: 0.8,0.9,1.9; xy","c: 1.8; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xy",error,ID
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.723,a1
1,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,0.785,a3
2,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.806,a5
3,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,0.862,a11
4,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,0.806,a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,3.5,4.0,4.0,4.0,4.0,3.5,3.5,3.0,3.0,4.0,...,4.0,3.5,4.0,3.5,3.5,4.0,4.0,3.0,0.723,v8
186,2.0,2.5,3.0,3.0,2.0,2.5,2.0,2.0,3.0,2.0,...,2.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,0.723,v9
187,1.0,2.0,1.5,1.5,2.0,1.5,1.0,1.5,1.0,2.0,...,1.5,1.0,1.0,1.5,1.0,1.5,2.0,1.5,0.723,v10
188,2.0,2.0,1.5,1.5,1.5,1.5,2.0,1.5,2.0,1.5,...,1.5,2.0,2.0,2.0,2.0,1.0,1.5,1.0,0.723,v11


In [17]:
lowest_mean, lowest_mean_idx, lowest_median, results_median = processing.get_lowest_error(results_sum)

In [18]:
lowest_mean_idx

Index(['c: 1.6; k: 0.2,0.3,1.3; xy'], dtype='object')